
### Statistical Classification
We want to classify survining the Titanic. This inference is based the set of features shown below. 

* $x_1$: Age
* $x_2$: Class
* $x_3$: Sex

The dataset will be split (80/20) into training and testing data. Two classification models will be used: logistic regression (parametric model) and k-nearest-neighbors (non-parametric model).

In [356]:
import numpy as np
import pandas as pd

In [357]:
TRAINING_TEST_SPLIT = 0.8
CLASSIFER_THRESHOLD = 0.5

data = pd.read_csv('data/titanic.csv')
data.dropna(inplace=True)
split_index = int(np.ceil(TRAINING_TEST_SPLIT * len(data)))
y = data['Survived']
X = data[['Sex', 'Age', 'Pclass']]

# Transforming text based binary data to intergers.
sex = pd.get_dummies(X['Sex'])
sex = sex['male']
X.drop(columns=['Sex'], inplace=True)
X.insert(1, 'Sex', sex)

# Adding bias feature.
constants = np.ones(len(X))
X.insert(0, 'consant', constants)

# numpy conversion.
y = np.array(y)
X = np.array(X)

# Split data into training and test (80/20)
training_X = X[0:split_index]
training_y = y[0:split_index]
test_X = X[split_index:]
test_y = y[split_index:]

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


### Logistic Regression

We begin by applying logistic regression. Logistic regression is a transformed version of linear regression, which limits the output to $[0,1]$, which represents a probability of being in a class or not. Note, we are only making inference on one class.

For logistic regression, we transform the $y_i$ using the Sigmoid function defined as: $f(x) \triangleq \frac{1}{1 + e^x}$. Thus we obtain the following model for our classification problem:
$$
\mathbb{P}({y_i}=1| \mathbf{x}, \mathbf{\theta}) = \frac{1}{1 + \exp \{\theta_0 + \theta_1 x_1 + \ldots + \theta_6 x_6\}} = \frac{1}{1 + \exp \{\mathbf{x}^{\text{T}}\mathbf{\theta}\}}
$$

The sample of observations $\{y_i\}_{i=1}^n$ can now be intepreted as a sequence of Bernoulli random variables. Thus we can obtain the following log-likelihood function rather easily:

$$
-\ell(\bold{X}, \bold{y} | \mathbf{\theta}) = \sum_{i=1}^n \big[(y_i-1)\mathbf{\theta}^{\text{T}}\mathbf{x}_i - \ln(1+\exp\{\mathbf{\theta}^{\text{T}}\mathbf{x}_i\})\big]
$$

Next, we fit the model to the training data using the log-likelihood function above. 

In [358]:
def sigmoid(x: np.array) -> float:
    return 1 / (1 + np.exp(-x))

In [359]:
def log_loss(params: np.array, y: np.array, X: np.array) -> float:
    assert(len(y) == len(X))
    n_observations = len(y)
    epsilon = 1e-5   
    log_loss = 0
    for i in range(0, n_observations):
        log_loss = log_loss + y[i] * np.log(sigmoid(params.T @ X[i])+epsilon) + (1 - y[i]) * np.log(1-sigmoid(params.T @ X[i])+epsilon)
    return log_loss

In [360]:
from scipy.optimize import minimize

# Solving minimzing loss function.
inital_solution = np.zeros(4)
solution = minimize(log_loss, x0=inital_solution, args=(training_y, training_X))

In [361]:
def classifier(data: np.array, params: np.array, threshold: float = CLASSIFER_THRESHOLD) -> int:
    prob = 1 / (1 + np.exp(params.T @ data))
    if prob > threshold:
        return 1
    else:
        return 0

In [362]:
def classify_data(data: np.array, params: np.array) -> np.array:
    n_observations = len(data)
    prediction = np.zeros(n_observations)
    for i in range(0, n_observations):
        prediction[i] = classifier(data[i], params)
    return prediction

In [363]:
def calculate_accuracy(prediction: np.array, validation: np.array) -> float:
    assert(len(prediction)==len(validation))
    return np.sum([1 for i in prediction-validation if i==0]) / len(validation)

In [364]:
prediction = classify_data(test_X, solution.x)
validation = test_y
accuracy = calculate_accuracy(prediction, validation)
print('Self-implemented logistic regression accuracy:', str(accuracy*100)[:4], '%')   

Self-implemented logistic regression accuracy: 72.2 %


### Comparison

To test the model accuracy two comparisons are made with models from sklearn. Firstly with sklearn's logistic regression package, and secondly with sklearn's kNN (k-nearest-neighbors) algoritm. In doing so we test both a parametric and a non-parametric model against the model implemented above.

In [365]:
from sklearn.linear_model import LogisticRegression

# sklearn logistic regression.
lr = LogisticRegression().fit(training_X, training_y)
prediction = lr.predict(test_X)
validation = test_y
accuracy = calculate_accuracy(prediction, validation)
print('Sklearn-implemented logistic regression accuracy:', str(accuracy*100)[:4], '%')   

Sklearn-implemented logistic regression accuracy: 75.0 %


In [366]:
from sklearn.neighbors import KNeighborsClassifier

# sklearn kNN algorithm.
knn = KNeighborsClassifier().fit(training_X, training_y)
prediction = knn.predict(test_X)
validation = test_y
accuracy = calculate_accuracy(prediction, validation)
print('Sklearn-implemented kNN accuracy:', str(accuracy*100)[:4], '%')   

Sklearn-implemented kNN accuracy: 66.6 %
